In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")
sys.path.append("IF-ConvTransformer-UbiComp2022-main/src/")
from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os
from thop import profile as profile_model
from torchlop import profile as profile_layer
from thop import clever_format
from models.model_builder import model_builder
from torchlop.rst_process import write_csv
from ptflops import get_model_complexity_info
from fvcore.nn import FlopCountAnalysis
# from model_summary import model_summary


import re

def find_percentage_before_MACs(text, substring):
    """
    Find the percentage value before the first occurrence of 'MACs' after a given substring.

    :param text: The text in which to search.
    :param substring: The substring after which to find the percentage.
    :return: The percentage found or a message if not found.
    """
    # Constructing the regular expression pattern
    pattern = rf"{re.escape(substring)}.*?(\d+\.\d+%).*?MACs"
    
    # Searching for the pattern in the text
    match = re.search(pattern, text)
    start = match.start()
    end = match.end()
    text = text[start:end]

    return extract_numbers_from_string(text)



import re

def extract_numbers_from_string(s):
    """
    Extract all numbers from the given string.

    :param s: The string to extract numbers from.
    :return: A list of numbers (as strings) extracted from the string.
    """
    # Regular expression pattern for matching numbers
    pattern = r'\d+\.\d+|\d+'

    # Finding all occurrences of the pattern in the string
    numbers = re.findall(pattern, s)

    return numbers


# DeepConvLSTM

In [4]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
args.f_in = 1
args.wavelet_filtering = False
args.model_type              = "deepconvlstm"#"deepconvlstm"#"sahar" #"deepconvlstm"

# args.cross_channel_interaction_type = "attn"
# args.cross_channel_aggregation_type = "FC"
# args.temporal_info_interaction_type = "lstm"
# args.temporal_info_aggregation_type = "tnaive"
args.input_length = int(128) # scoda 196   motionsense 128  ucihar 128
args.num_classes = 6 #scoda 10   motionsense 6   ucihar 6 
args.c_in       = 12  #scoda 30   motionsense 12   ucihar  9
args.filter_scaling_factor = 1
model = model_builder(args)
#model.double()

Build the DeepConvLSTM model!


In [5]:
macs, params = get_model_complexity_info(model, (1, args.input_length, args.c_in), as_strings=True,
                                       print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

model_builder(
  522.57 k, 100.000% Params, 138.36 MMac, 99.739% MACs, 
  (model): DeepConvLSTM(
    522.57 k, 100.000% Params, 138.36 MMac, 99.739% MACs, 
    (conv_blocks): ModuleList(
      62.02 k, 11.868% Params, 86.73 MMac, 62.515% MACs, 
      (0): ConvBlock(
        20.93 k, 4.005% Params, 30.34 MMac, 21.872% MACs, 
        (conv1): Conv2d(384, 0.073% Params, 571.39 KMac, 0.412% MACs, 1, 64, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(0, 0.000% Params, 187.39 KMac, 0.135% MACs, inplace=True)
        (conv2): Conv2d(20.54 k, 3.931% Params, 29.58 MMac, 21.325% MACs, 64, 64, kernel_size=(5, 1), stride=(1, 1))
      )
      (1): ConvBlock(
        41.09 k, 7.863% Params, 56.38 MMac, 40.643% MACs, 
        (conv1): Conv2d(20.54 k, 3.931% Params, 28.6 MMac, 20.614% MACs, 64, 64, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(0, 0.000% Params, 175.1 KMac, 0.126% MACs, inplace=True)
        (conv2): Conv2d(20.54 k, 3.931% Params, 27.61 MMac, 19.903% MACs, 64, 64, k

# SHHAR

In [24]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
args.f_in = 1
args.wavelet_filtering = False
args.model_type              = "sahar"#"deepconvlstm"#"sahar" #"deepconvlstm"

# args.cross_channel_interaction_type = "attn"
# args.cross_channel_aggregation_type = "FC"
# args.temporal_info_interaction_type = "lstm"
# args.temporal_info_aggregation_type = "tnaive"
args.input_length = 128
args.num_classes = 12
args.c_in       = 9
args.filter_scaling_factor = 1
model = model_builder(args)
model.double();

Build the sahar model!


In [25]:
macs, params = get_model_complexity_info(model, (1, args.input_length, args.c_in), as_strings=True,
                                       print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

model_builder(
  424.08 k, 100.000% Params, 56.67 MMac, 99.191% MACs, 
  (model): SA_HAR(
    424.08 k, 100.000% Params, 56.67 MMac, 99.191% MACs, 
    (first_conv): ConvBlock(
      1.67 k, 0.393% Params, 2.07 MMac, 3.621% MACs, 
      (conv1): Conv2d(768, 0.181% Params, 884.74 KMac, 1.549% MACs, 1, 128, kernel_size=(5, 1), stride=(1, 1), padding=same)
      (relu): ReLU(0, 0.000% Params, 148.61 KMac, 0.260% MACs, inplace=True)
      (conv2): Conv2d(641, 0.151% Params, 738.43 KMac, 1.292% MACs, 128, 1, kernel_size=(5, 1), stride=(1, 1), padding=same)
      (norm1): BatchNorm2d(256, 0.060% Params, 294.91 KMac, 0.516% MACs, 128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (norm2): BatchNorm2d(2, 0.000% Params, 2.3 KMac, 0.004% MACs, 1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (SensorAttention): SensorAttention(
      1.43 k, 0.336% Params, 1.77 MMac, 3.101% MACs, 
      (ln): LayerNorm(18, 0.004% Params, 1.15 KMac, 0.002% MACs, 

In [ ]:
first_conv
SensorAttention

# Tiny HAR

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
args.f_in = 1
args.wavelet_filtering = False
args.model_type              = "tinyhar"#"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"
args.input_length = int(128) # scoda 196   motionsense 128  ucihar 128
args.num_classes = 6 #scoda 10   motionsense 6   ucihar 6 
args.c_in       = 9  #scoda 30   motionsense 12   ucihar  9
args.filter_scaling_factor = 1
model = model_builder(args)

64
Build the TinyHAR model!


In [3]:
macs, params = get_model_complexity_info(model, (1, args.input_length, args.c_in), as_strings=True,
                                       print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

model_builder(
  298.18 k, 99.999% Params, 35.84 MMac, 98.819% MACs, 
  (model): TinyHAR_Model(
    298.18 k, 99.999% Params, 35.84 MMac, 98.819% MACs, 
    (layers_conv): ModuleList(
      62.53 k, 20.970% Params, 27.14 MMac, 74.847% MACs, 
      (0): Sequential(
        512, 0.172% Params, 642.82 KMac, 1.773% MACs, 
        (0): Conv2d(384, 0.129% Params, 428.54 KMac, 1.182% MACs, 1, 64, kernel_size=(5, 1), stride=(1, 1))
        (1): ReLU(0, 0.000% Params, 71.42 KMac, 0.197% MACs, inplace=True)
        (2): BatchNorm2d(128, 0.043% Params, 142.85 KMac, 0.394% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        20.67 k, 6.933% Params, 11.2 MMac, 30.876% MACs, 
        (0): Conv2d(20.54 k, 6.890% Params, 11.09 MMac, 30.590% MACs, 64, 64, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(0, 0.000% Params, 34.56 KMac, 0.095% MACs, inplace=True)
        (2): BatchNorm2d(128, 0.043% Params, 69.12 KMac, 0.191% MACs, 64, eps=1e-

# Attend

In [11]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
args.f_in = 1
args.wavelet_filtering = False
args.model_type = "attend"
args.input_length = int(128) # scoda 196   motionsense 128  ucihar 128
args.num_classes = 6 #scoda 10   motionsense 6   ucihar 6 
args.c_in       = 9  #scoda 30   motionsense 12   ucihar  9
args.filter_scaling_factor = 0.30
model = model_builder(args)

Build the AttendDiscriminate model!


In [12]:
macs, params = get_model_complexity_info(model, (1, args.input_length, args.c_in), as_strings=True,
                                       print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

model_builder(
  35.56 k, 99.997% Params, 1.05 MMac, 97.012% MACs, 
  (model): AttendDiscriminate(
    35.56 k, 99.997% Params, 1.05 MMac, 97.012% MACs, 
    (fe): FeatureExtractor(
      35.32 k, 99.339% Params, 1.05 MMac, 96.991% MACs, 
      (conv1): Conv2d(114, 0.321% Params, 49.48 KMac, 4.585% MACs, 1, 19, kernel_size=(5, 1), stride=(2, 1))
      (conv2): Conv2d(1.82 k, 5.130% Params, 370.27 KMac, 34.315% MACs, 19, 19, kernel_size=(5, 1), stride=(2, 1))
      (conv3): Conv2d(1.82 k, 5.130% Params, 165.98 KMac, 15.383% MACs, 19, 19, kernel_size=(5, 1), stride=(2, 1))
      (conv4): Conv2d(1.82 k, 5.130% Params, 114.91 KMac, 10.650% MACs, 19, 19, kernel_size=(5, 1), stride=(1, 1))
      (activation): ReLU(0, 0.000% Params, 15.03 KMac, 1.393% MACs, )
      (dropout): Dropout(0, 0.000% Params, 0.0 Mac, 0.000% MACs, p=0.5, inplace=False)
      (rnn): GRU(28.61 k, 80.474% Params, 262.31 KMac, 24.310% MACs, 133, 38, num_layers=2, dropout=0.25)
      (ta): TemporalAttention(
        39, 0

In [6]:
# model_summary(model, (1, args.input_length, args.c_in), query_granularity=1)

In [5]:
# thop
input = torch.randn(1, 1, args.input_length, args.c_in)
macs, params = profile_model(model, inputs=(input, ), ret_layer_info=False)
macs, params = clever_format([macs, params], "%.3f")
print(macs,params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_gru() for <class 'torch.nn.modules.rnn.GRU'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_softmax() for <class 'torch.nn.modules.activation.Softmax'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
323.253M 519.885K


In [ ]:
profile_model

In [16]:
# torchlop
macs, params, layer_infos = profile_layer(model, inputs=(input, ))

csv_file='profile_2.csv'
write_csv(csv_file,layer_infos)

In [4]:
# fvcore
flops = FlopCountAnalysis(model, input)

In [5]:
flops.total()

Unsupported operator aten::softmax encountered 241 time(s)
Unsupported operator aten::mul encountered 241 time(s)
Unsupported operator aten::add encountered 240 time(s)
Unsupported operator aten::gru encountered 1 time(s)
Unsupported operator aten::sum encountered 1 time(s)


220709376

In [8]:
flops.by_module_and_operator()

{'': Counter({'conv': 216253440, 'bmm': 4423680, 'linear': 32256}),
 'model': Counter({'conv': 216253440, 'bmm': 4423680, 'linear': 32256}),
 'model.fe': Counter({'conv': 216253440, 'bmm': 4423680, 'linear': 30720}),
 'model.fe.conv1': Counter({'conv': 967680}),
 'model.fe.conv2': Counter({'conv': 60948480}),
 'model.fe.conv3': Counter({'conv': 59965440}),
 'model.fe.conv4': Counter({'conv': 58982400}),
 'model.fe.activation': Counter(),
 'model.fe.dropout': Counter(),
 'model.fe.rnn': Counter(),
 'model.fe.ta': Counter({'linear': 30720}),
 'model.fe.ta.fc': Counter({'linear': 30720}),
 'model.fe.ta.sm': Counter(),
 'model.fe.sa': Counter({'conv': 35389440, 'bmm': 4423680}),
 'model.fe.sa.query': Counter({'conv': 11796480}),
 'model.fe.sa.key': Counter({'conv': 11796480}),
 'model.fe.sa.value': Counter({'conv': 11796480}),
 'model.dropout': Counter(),
 'model.classifier': Counter({'linear': 1536}),
 'model.classifier.fc': Counter({'linear': 1536})}

In [9]:
from fvcore.nn import flop_count_table
print(flop_count_table(flops))

| module                 | #parameters or shape   | #flops     |
|:-----------------------|:-----------------------|:-----------|
| model                  | 0.52M                  | 0.221G     |
|  fe                    |  0.518M                |  0.221G    |
|   fe.conv1             |   0.384K               |   0.968M   |
|    fe.conv1.weight     |    (64, 1, 5, 1)       |            |
|    fe.conv1.bias       |    (64,)               |            |
|   fe.conv2             |   20.544K              |   60.948M  |
|    fe.conv2.weight     |    (64, 64, 5, 1)      |            |
|    fe.conv2.bias       |    (64,)               |            |
|   fe.conv3             |   20.544K              |   59.965M  |
|    fe.conv3.weight     |    (64, 64, 5, 1)      |            |
|    fe.conv3.bias       |    (64,)               |            |
|   fe.conv4             |   20.544K              |   58.982M  |
|    fe.conv4.weight     |    (64, 64, 5, 1)      |            |
|    fe.conv4.bias       

In [ ]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
# TODO change the path as relative path
args.to_save_path     = "../../../ISWC2022LearnableFilter/Run_logs"              
args.freq_save_path   = "../../../ISWC2022LearnableFilter/Freq_data"
args.window_save_path = "../../../ISWC2022LearnableFilter/Sliding_window"
args.root_path        = "../../../datasets"


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 256                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 5
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"